### Youtube Sentence Adjust Word List Ratio

In [ ]:
import pandas as pd
import numpy as np
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import glob

In [ ]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

In [ ]:
folder_name = "Turkish"
word_num = 28

#### Calculate Sentence Ratio By Word

In [ ]:
df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_adjust_word = df_word.iloc[0:word_num,]
df_adjust_word

In [ ]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{folder_name.lower().capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
#df_youtube_sentence = df_youtube_sentence.head(40000)
df_youtube_sentence.reset_index(inplace=True)
df_youtube_sentence

In [ ]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(df_adjust_word["word"].to_list())
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [ ]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id",f"{word_num}_word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

In [ ]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

In [ ]:
df_sentence_ratio_result.to_csv(f"Youtube_Sentence_{word_num}_Word_Ratio_Result.csv", index=False)

#### Calculate Videoid Ratio By Word

In [ ]:
df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_adjust_word = df_word.iloc[0:word_num,]
df_adjust_word

In [ ]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{folder_name.lower().capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
#df_youtube_sentence = df_youtube_sentence.head(40000)
df_youtube_sentence.reset_index(inplace=True)
df_youtube_sentence

In [ ]:
df_videoid_sentence = df_youtube_sentence.groupby("video_id")["sentence"].apply(" ".join).reset_index()
df_videoid_sentence

In [ ]:
def videoid_word_ratio(df_word, df_videoid_sentence, word_column_name, videoid_sentence_column_name, word_num):
    word_set = set(df_word[f"{word_column_name}"].to_list())
    for i in range(len(df_videoid_sentence)):
        text = df_videoid_sentence.loc[i,f"{videoid_sentence_column_name}"]
        text_word = re.findall(r"\w+", text, re.UNICODE)
        text_word_set = set(text_word)
        intersect_word = list(word_set.intersection(text_word_set))
        different_word = list(text_word_set.difference(word_set))
        df_videoid_sentence.loc[i,f"{word_num}_word_ratio"] = round(((len(intersect_word)/len(text_word)+0.001)*100),)
        df_videoid_sentence.loc[i,"different_word"] = ", ".join(different_word)
        df_videoid_sentence.loc[i,"intersect_word"] = ", ".join(intersect_word)
        
    return df_videoid_sentence

In [ ]:
df_text_ratio_result = videoid_word_ratio(df_adjust_word, df_videoid_sentence, "word", "sentence", word_num)
df_text_ratio_result

In [ ]:
df_text_ratio_result["different_word"] = df_text_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result["intersect_word"] = df_text_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result 

In [ ]:
df_text_ratio_result.to_csv(f"Youtube_Videoid_Text_{word_num}_Word_Ratio_Result.csv", index=False)

#### Select Adjust Sentence Ratio Result

In [ ]:
df_sentence_ratio_result = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/5-Youtube Sentence Adjust Words Ratio/Youtube_Sentence_{word_num}_Word_Ratio_Result.csv")
#df_sentence_ratio_result = pd.read_csv(f"Youtube_Sentence_{word_num}_Word_Ratio_Result.csv")
df_sentence_ratio_result

In [ ]:
adjust_word_ratio = 100
df_adjust_ratio = df_sentence_ratio_result[df_sentence_ratio_result[f"{word_num}_word_ratio"] >= adjust_word_ratio]
df_adjust_ratio.reset_index(inplace=True, drop=True)
df_adjust_ratio

In [ ]:
df_adjust_ratio.to_csv(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Sentences.csv", index=False)

In [ ]:
df_adjust_ratio.to_excel(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Sentences.xlsx", index=False)

#### Calculate Sequential Sentence Length Of Adjust Ratio Result

In [ ]:
df_sentence_ratio_result = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/5-Youtube Sentence Adjust Words Ratio/Youtube_Sentence_{word_num}_Word_Ratio_Result.csv")
#df_sentence_ratio_result = pd.read_csv(f"Youtube_Sentence_{word_num}_Word_Ratio_Result.csv")
df_sentence_ratio_result

In [ ]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].astype("object")
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].astype("object")

In [ ]:
df_adjust_ratio = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/5-Youtube Sentence Adjust Words Ratio/Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Sentences.csv")
#df_adjust_ratio = pd.read_csv(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Sentences.csv")
df_adjust_ratio

In [ ]:
df_adjust_ratio["different_word"] = df_adjust_ratio["different_word"].astype("object")
df_adjust_ratio["intersect_word"] = df_adjust_ratio["intersect_word"].astype("object")

In [ ]:
df_adjust_ratio["with_word"] = 1
df_adjust_ratio

In [ ]:
df_adjust_ratio_flag = pd.merge(df_sentence_ratio_result , df_adjust_ratio, how="left", on=["index","sentence","start_time","end_time","video_id",f"{word_num}_word_ratio","different_word","intersect_word"])
df_adjust_ratio_flag["with_word"].fillna(0, inplace=True)
df_adjust_ratio_flag

In [ ]:
df_adjust_ratio_flag.with_word.value_counts()

In [ ]:
# Code Running For Video_id Sentence
zeros = []
ones = []
zero_list = []
one_list = []
for i in range(len(df_adjust_ratio_flag)):
    videoid = df_adjust_ratio_flag.loc[i,"video_id"]    
    if df_adjust_ratio_flag.loc[i,"with_word"] == 0:
        zeros.append(i)
        try:
            one_list.append((ones[0],len(ones)))
        except:
            pass
        ones = []
    else:
        ones.append(i)
        try:
            zero_list.append((zeros[0],len(zeros)))
        except:
            pass
        zeros = []

    try:
        if df_adjust_ratio_flag.loc[i+1,"video_id"] != videoid:
            try:
                one_list.append((ones[0],len(ones)))
                ones = []
            except:
                pass
            try:
                zero_list.append((zeros[0],len(zeros)))
                zeros = []
            except:
                pass
        else:
            pass
    except:
        pass

try:
    one_list.append((ones[0],len(ones)))
except:
    pass
try:
    zero_list.append((zeros[0],len(zeros)))
except:
    pass

In [ ]:
df_one = pd.DataFrame(one_list)
df_one.rename(columns={0:"index",1:"length"}, inplace=True)
df_one.drop_duplicates(inplace=True)
df_one.sort_values(by="length", ascending=False, inplace=True)
df_one.reset_index(drop=True, inplace=True)
df_one

In [ ]:
df_one_all = pd.DataFrame()
for i in range(len(df_one)):
    index = df_one.loc[i,"index"]
    length = df_one.loc[i,"length"]
    start_index = index
    end_index = index+length
    df_video = df_adjust_ratio_flag.iloc[start_index:end_index,]
    df_one_all = pd.concat([df_one_all, df_video],axis=0)
df_one_all.reset_index(inplace=True, drop=True)
df_one_all

In [ ]:
#df_one_all.to_excel(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Sequential_Sentences_And_Others.xlsx", index=False)

In [ ]:
# Only Sequential Sentences
df_one_sequential = df_one[df_one.length > 1]
df_one_sequential

In [ ]:
df_one_sequential_all = pd.DataFrame()
for i in range(len(df_one_sequential)):
    index = df_one_sequential.loc[i,"index"]
    length = df_one_sequential.loc[i,"length"]
    start_index = index
    end_index = index+length
    df_video = df_adjust_ratio_flag.iloc[start_index:end_index,]
    df_one_sequential_all = pd.concat([df_one_sequential_all, df_video],axis=0)
df_one_sequential_all.reset_index(inplace=True, drop=True)
df_one_sequential_all

In [ ]:
df_one_sequential_all.to_csv(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Only_Sequential_Sentences.csv", index=False)

In [ ]:
df_one_sequential_all.to_excel(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Only_Sequential_Sentences.xlsx", index=False)

#### Count Sentence Words By Adjust Column

In [ ]:
df_one_sequential_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/5-Youtube Sentence Adjust Words Ratio/Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Only_Sequential_Sentences.csv")
#df_one_sequential_all = pd.read_csv(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Only_Sequential_Sentences.csv")
df_one_sequential_all

In [ ]:
def text_to_list_df(df_target, target_column): # df_target include dict value columns, target_column occurs dict
    text_value_list = []
    for i in df_target[f"{target_column}"]:
        try:
            var = re.findall(r"\w+", i)
            for j in var:
                text_value_list.append(j)
        except:
            pass
    df_alone_value = pd.DataFrame(text_value_list)
    df_alone_value.rename(columns={0:f"{target_column}_alone"}, inplace=True)
    df_unique_count = pd.DataFrame(df_alone_value.value_counts(ascending=False))
    df_unique_count.reset_index(inplace=True)
    df_unique_count.rename(columns={0:"count"}, inplace=True)
            
    return df_unique_count

In [ ]:
df_unique_word_count = text_to_list_df(df_one_sequential_all, "intersect_word")
df_unique_word_count

In [ ]:
#df_unique_word_count.to_excel(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Sequential_Sentence_Words_Count.xlsx", index=False)

#### Count Adjust Sentence Ratio Result By Videoid

In [ ]:
df_adjust_ratio = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/5-Youtube Sentence Adjust Words Ratio/Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Sentences.csv")
#df_adjust_ratio = pd.read_csv(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Sentences.csv")
df_adjust_ratio

In [ ]:
df_adjust_ratio["different_word"] = df_adjust_ratio["different_word"].astype("object")
df_adjust_ratio["intersect_word"] = df_adjust_ratio["intersect_word"].astype("object")

In [ ]:
df_ratio_count = pd.DataFrame(df_adjust_ratio.groupby("video_id")[f"{word_num}_word_ratio"].count())
df_ratio_count.reset_index(inplace=True)
df_ratio_count.rename(columns={f"{word_num}_word_ratio":f"{word_num}_word_ratio_count"}, inplace=True)
df_ratio_count.sort_values(by=f"{word_num}_word_ratio_count", ascending=False, inplace=True)
df_ratio_count.reset_index(drop=True,inplace=True)
df_ratio_count

In [ ]:
#df_ratio_count.to_excel(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Videoid_Count.xlsx", index=False)

#### Find Word In Sentence Time Interval

In [ ]:
df_word_count_low = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/{level.lower().capitalize()}/3-Lesson Level Result/Lesson_{level.lower().capitalize()}_Low_Count_Words.xlsx")
df_word_count_low

In [ ]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{folder_name.lower().capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
#df_youtube_sentence = df_youtube_sentence.head(40000)
df_youtube_sentence

In [ ]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [ ]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

In [ ]:
word_list = df_word_count_low.iloc[:,0].values.tolist()

In [ ]:
df_word_result = pd.DataFrame()
for i in word_list:
    try:
        word_in_video = df_youtube_sentence[df_youtube_sentence.sentence.str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].sample(1)
    except:
        word_in_video = df_youtube_sentence[df_youtube_sentence.sentence.str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(1) 
    word_in_video.insert(0,"word",i)
    df_word_result = pd.concat([df_word_result,word_in_video], axis=0)
df_word_result.reset_index(drop=True, inplace=True)
df_word_result = df_word_result.sort_values(by=["video_id","start_time"], ascending=True)
df_word_result.reset_index(drop=True, inplace=True)
df_word_result

In [ ]:
word_time_loc_list = []
def word_time_loc(df):
    for i in range(len(df)):
        word = df.loc[i,"word"]
        start_time = df.loc[i,"start_time"]
        end_time = df.loc[i,"end_time"]
        sentence = df.loc[i,"sentence"]
        video_id = df.loc[i,"video_id"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=["word","start_time","end_time","sentence","video_id"])

    return df_word_time_loc

In [ ]:
df_word_time_loc_result = word_time_loc(df_word_result)
df_word_time_loc_result

In [ ]:
time_shift = 0.3
df_word_time_loc_result.start_time = df_word_time_loc_result.start_time.apply(lambda x: (x-time_shift))
df_word_time_loc_result.end_time = df_word_time_loc_result.end_time.apply(lambda x: (x+time_shift))
df_word_time_loc_result

In [ ]:
#df_word_time_loc_result.to_excel(f"Youtube_Sentence_{word_num}_Low_Word_Time_Interval.xlsx", index=False)